# Notebook experiment for model training (finally)

In [1]:
%pwd

'/Users/soogeunpark/Documents/text_summarizer_cicd/Text-Summarizer/research'

In [2]:
import os
os.chdir("..")

In [3]:
os.getcwd()

'/Users/soogeunpark/Documents/text_summarizer_cicd/Text-Summarizer'

First I need to update the ```config.yaml``` and ```params.yaml``` files.

Now will set up the ```entity```.

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    """
    Providing the format of the model trainer config
    """
    # first format for the parameters in the config.yaml
    root_dir: Path
    data_path: Path
    model_ckpk: Path # model_ckpk is also a path: we get it from google
    
    # now the parameters in param.yaml
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

Now writing the ```config/configuration.py: configuration manager```

In [14]:

from TextSummarizer.constants import *
# the asterisk imports everything in the directory
# the 'constants' includes the CONFIG_FILE_PATH and PARAMS_FILE_PATH

from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        """
        This class is used to manage the configuration of the project
        """

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots]) 
        # this refers to the artifacts_roots in the config.yaml
        # this creates the 'artifacts' directory
        # because self.config does the "read_yaml" function which uses ConfigBox, the artifacts_roots can be just accessed by using the dot notation

    def model_trainer_config(self) -> ModelTrainerConfig:
        """
        This function returns the model trainer config
        """
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpk = config.model_ckpk,
            
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )


Now working on the components: this is the step that we are really defining the actions conducted by the ```ModelTrainer``` stage:

First, import libraries needed for training the model:

In [1]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


In [ ]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer

class ModelTrainer:
    def __init__(self, config: DataTransformationConfig):
        """
        This class is used to execute data validation
        """
        self.config = config
        
        # also I initialize the tokenizer: I need to load the tokenizer from the pretrained model
        # this could be the model name itself from google, or the saved model after training
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    # now I need the function that convert data ('examples') into features ('tokens')
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], 
                                         max_length = 1024,
                                         truncation = True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'],
                                              max_length = 128,
                                              truncation = True)
            
        return{
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    # this method then executes the data transformation
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        logger.info(f"Dataset loaded from {self.config.data_path}.")
        
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        # batched processing instead of individual observation unit processing, to allow more efficient processing
        
        logger.info(f"Dataset converted to features successfully.")
        
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, self.config.result_file_name))
        
        logger.info(f"Dataset saved at {os.path.join(self.config.root_dir, self.config.result_file_name)}.")